In [ ]:
import os
import sys
import random
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
im_width = 512
im_height = 512
im_chan = 1

path_train = '/kaggle/input/small-dataset/Train/'
path_test = '/kaggle/input/small-dataset/Test/'

train_ids = next(os.walk(path_train+"images"))[2]
test_ids = next(os.walk(path_test+"images"))[2]

In [ ]:
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=bool)

print('Getting and resizing train images and masks ... ')

sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img = load_img(path + '/images/' + id_)
    
    x = img_to_array(img)[:,:,1]
    x = np.expand_dims(x, axis=-1)
    X_train[n] = x
    
    
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    mask = np.expand_dims(mask, axis=-1)
    Y_train[n] = mask   
    

print('Done!')

In [ ]:
ix = random.randint(0, len(train_ids))
plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.imshow(np.dstack((X_train[ix],X_train[ix],X_train[ix])))
plt.title("Image")

plt.subplot(1, 3, 2)
tmp = np.squeeze(Y_train[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.title("Ground truth")

plt.show()

In [ ]:
from tensorflow.keras import metrics
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
class MeanIoUMetric(metrics.Metric):
    
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super(MeanIoUMetric, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes   
        self.iou_metric = metrics.MeanIoU(num_classes=num_classes) 

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_ = tf.cast(y_pred > 0.5, tf.int32) 
        self.iou_metric.update_state(y_true, y_pred_)

    def result(self):
        return self.iou_metric.result()

    def reset_state(self):
        self.iou_metric.reset_state()
        
    @classmethod
    def from_config(cls, config):
        return cls(**config)  

    def get_config(self):
        config = super().get_config()
        config.update({"num_classes": self.num_classes}) 
        return config
    
    
mean_iou_metric = MeanIoUMetric(num_classes=2)

In [ ]:
# Build U-Net model
inputs = Input((im_height, im_width, im_chan))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=[ mean_iou_metric])
model.summary()

In [ ]:
earlystopper = EarlyStopping(patience=10, verbose=2)
checkpointer = ModelCheckpoint('UNET.keras', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.2, batch_size=16, epochs=150, 
                    callbacks=[earlystopper, checkpointer])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

plt.figure(figsize=(10, 5))  
plt.style.use('seaborn-whitegrid')

plt.plot(results.history['loss'], label='Training Loss', color='#1f77b4', linewidth=2)
plt.plot(results.history['val_loss'], label='Validation Loss', color='#ff7f0e', linestyle='--', linewidth=2)
plt.plot(results.history['mean_iou'], label='Mean IoU', color='#2ca02c', linewidth=2)
plt.plot(results.history['val_mean_iou'], label='Validation Mean IoU', color='#d62728', linestyle='--', linewidth=2)

plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss / Mean IoU', fontsize=12)
plt.title('Training and Validation Metrics', fontsize=14, fontweight='bold') 

plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7) 
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(loc='upper left', fontsize=10, frameon=True, facecolor='white', framealpha=0.9) 

# plt.ylim(0, 1)

plt.savefig('UNET.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Get and resize test images and masks
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), im_height, im_width, 1), dtype=bool) 



print('Getting and resizing test images and masks ... ')
sys.stdout.flush()

for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = np.expand_dims(x, axis=-1)
    X_test[n] = x

    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    mask = np.expand_dims(mask, axis=-1)
    Y_test[n] = mask

print('Done!')

In [ ]:
# Predict on Test Dataset
model = tf.keras.models.load_model(
    '/kaggle/working/UNET.keras',
    custom_objects={'MeanIoUMetric': MeanIoUMetric} ,safe_mode=False
)

preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)



In [ ]:
#Perform a sanity check on some random Test Samples
import matplotlib.pyplot as plt
import numpy as np
import random

 
plt.style.use('seaborn-whitegrid') 

fig, axes = plt.subplots(3, 3, figsize=(15, 12))  
fig.subplots_adjust(hspace=0.3, wspace=-0.5)

for i in range(3):
    if i ==0 :
        axes[i, 0].set_title("Image", fontsize=16) 
        axes[i, 1].set_title("Ground Truth", fontsize=16)
        axes[i, 2].set_title("Predicted", fontsize=16)
        
        
    ix = random.randint(0, len(preds_test_t) - 1) 

    # Image
    axes[i, 0].imshow(np.dstack((X_test[ix], X_test[ix], X_test[ix])))
    
    axes[i, 0].axis('off') 

    
    # Ground Truth
    im_gt = axes[i, 1].imshow(Y_test[ix], cmap='viridis')
    
    axes[i, 1].axis('off')
    

    
    # Prediction
    im_pred = axes[i, 2].imshow(preds_test_t[ix], cmap='viridis')
    
    axes[i, 2].axis('off')
    

plt.savefig("prediction_visualization.png", dpi=600) 
plt.show()